In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import numpy as np
from nltk.corpus import stopwords #provides list of english stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#PRINT VERSION!!!
tf.__version__

'2.6.0'

In [5]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/Text_Summarization/Reviews.csv')


In [6]:
train.head()

Id  ...                                               Text
0   1  ...  I have bought several of the Vitality canned d...
1   2  ...  Product arrived labeled as Jumbo Salted Peanut...
2   3  ...  This is a confection that has been around a fe...
3   4  ...  If you are looking for the secret ingredient i...
4   5  ...  Great taffy at a great price.  There was a wid...

[5 rows x 10 columns]

In [7]:
# We only need the last two columns for the text_summarization task
train = train[['Summary','Text']]

In [8]:
# We create a new column to indicate the length of 
# the sentence
train['text_length'] = train['Text'].str.count(' ')

In [11]:
train['text_length']

0         48
1         30
2         98
3         42
4         29
          ..
568449    25
568450    45
568451    70
568452    36
568453    20
Name: text_length, Length: 568454, dtype: int64

In [13]:
# Another column to see the lengths of respective summaries
train['summary_length'] = train['Summary'].str.count(' ')
train['summary_length']
# Shows that we have a small summary length.

0         3.0
1         2.0
2         3.0
3         1.0
4         1.0
         ... 
568449    3.0
568450    0.0
568451    3.0
568452    4.0
568453    1.0
Name: summary_length, Length: 568454, dtype: float64

In [14]:
# Filtering out and using rows with small text and summary lengths.
train = train.loc[train['summary_length']<8]
train = train.loc[train['text_length']<30]

In [15]:
print(train.shape)


(109053, 4)


In [17]:
train.head()

Summary  ... summary_length
4              Great taffy  ...            1.0
7   Wonderful, tasty taffy  ...            2.0
8               Yay Barley  ...            1.0
9         Healthy Dog Food  ...            2.0
13       fresh and greasy!  ...            2.0

[5 rows x 4 columns]

In [19]:
# Convert to lower case
train['text_lower'] = train['Text'].str.lower()
# Remove punctuation
train['text_no_punctuation'] = train['text_lower'].str.replace('[^\w\s]','')

# We will not remove stop-words because they are needed to understand
# the sequence of words for building summary.
# If need be remove stopwords using the following code:
#train['english_no_stopwords'] = train['english_no_punctuation'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#train["english_no_stopwords"] = train["english_no_stopwords"].fillna("fillna")
#train["english_no_stopwords"] = train["english_no_stopwords"] 

In [18]:
# Convert summary to lower case.
train['summary_lower'] = train["Summary"].str.lower()

# add start and end token to each summary
train['summary_no_punctuation'] =  '_start_' + ' ' +train['summary_lower'].str.replace('[^\w\s]','')+ ' ' +'_end_'

### MODEL BUILDING:

In [21]:
max_features1 = 5000
maxlen1 = 30 # for text

max_features2 = 5000
maxlen2 = 8 #for summary

In [22]:
# Tokenize the text
tok1 = tf.keras.preprocessing.text.Tokenizer(num_words=max_features1) 
#fit to cleaned text
tok1.fit_on_texts(list(train['text_no_punctuation'].astype(str))) 
tf_train_text =tok1.texts_to_sequences(list(train['text_no_punctuation'].astype(str)))
#let's execute pad step
tf_train_text =tf.keras.preprocessing.sequence.pad_sequences(tf_train_text, maxlen=maxlen1)  

In [23]:
# Tokenize the summaries
tok2 = tf.keras.preprocessing.text.Tokenizer(num_words=max_features2, filters = '*') 
#fit to cleaned summaries
tok2.fit_on_texts(list(train['summary_no_punctuation'].astype(str)))
tf_train_summary = tok2.texts_to_sequences(list(train['summary_no_punctuation'].astype(str)))
tf_train_summary = tf.keras.preprocessing.sequence.pad_sequences(tf_train_summary, maxlen=maxlen2, padding ='post') 

### Define Model Architecture

In [24]:
# creating new variable for vectorized summary.
vectorized_summary = tf_train_summary
# For Decoder Input, you don't need the last word as that is only for prediction
# when we are training using Teacher Forcing.
decoder_input_data = vectorized_summary[:, :-1]

# Decoder Target Data Is Ahead By 1 Time Step From Decoder Input Data (Teacher Forcing)
decoder_target_data = vectorized_summary[:, 1:]

print(f'Shape of decoder input: {decoder_input_data.shape}')
print(f'Shape of decoder target: {decoder_target_data.shape}')

vectorized_text = tf_train_text
# Encoder input is simply the body of the issue text
encoder_input_data = vectorized_text
doc_length = encoder_input_data.shape[1]
print(f'Shape of encoder input: {encoder_input_data.shape}')


Shape of decoder input: (109053, 7)
Shape of decoder target: (109053, 7)
Shape of encoder input: (109053, 30)


In [25]:
# define the vocab size
# +1 is for unk tokens
vocab_size_encoder = len(tok1.word_index) + 1
vocab_size_decoder = len(tok2.word_index) + 1

NOTE--> Why to add 1?
Because Tokenizer.word_index is a python dictionary that contains token keys (string) and token ID values (integer), and where the first token ID is 1 (not zero) and where the token IDs are assigned incrementally.  Therefore, we need vocabulary of size len(word_index) + 1 to be able to index up to the greatest token ID.

In [26]:
#arbitrarly set dimension for embedding and # of hidden units
latent_dim = 50
# It is a hyper-parameter.



##### ENCODER ARCHITECTURE:

In [27]:
encoder_inputs = tf.keras.Input(shape=(doc_length,), name='Encoder-Input')

# Word embeding layer for encoder (English text)
x = tf.keras.layers.Embedding(vocab_size_encoder, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)


#Batch normalization layer is used so that the distribution of the inputs 
#to a specific layer doesn't change over time
x = tf.keras.layers.BatchNormalization(name='Encoder-Batchnorm-1')(x)


# We do not need the `encoder_output` just the hidden state.
_, state_h = tf.keras.layers.GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

# Encapsulate the encoder as a separate entity so we can just 
#  encode without decoding if we want to.
encoder_model = tf.keras.Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)



##### DECODER ARCHITECTURE

In [28]:
# for teacher forcing
decoder_inputs = tf.keras.Input(shape=(None,), name='Decoder-Input')

# Word Embedding For Decoder (Italian text)
dec_emb = tf.keras.layers.Embedding(vocab_size_decoder, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
#again batch normalization
dec_bn = tf.keras.layers.BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

# Set up the decoder, using `decoder_state_input` as initial state.
decoder_gru = tf.keras.layers.GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out) #the decoder "decodes" the encoder output.
x = tf.keras.layers.BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

# Dense layer for prediction
decoder_dense = tf.keras.layers.Dense(vocab_size_decoder, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)



##### BUILDING A SEQ2SEQ MODEL

In [33]:
seq2seq_Model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

seq2seq_Model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

** Examine Model Architecture Summary **

In [34]:
#from seq2seq_utils import viz_model_architecture
seq2seq_Model.summary()
#viz_model_architecture(seq2seq_Model)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Decoder-Word-Embedding (Embeddi (None, None, 50)     692150      Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      [(None, 30)]         0                                            
__________________________________________________________________________________________________
Decoder-Batchnorm-1 (BatchNorma (None, None, 50)     200         Decoder-Word-Embedding[0][0]     
____________________________________________________________________________________________

# Train Model

In [35]:
batch_size = 64
epochs = 2 
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,  epochs=epochs ,  validation_split=0.12) 

Epoch 1/2
1500/1500 [==============================] - 271s 177ms/step - loss: 2.6436 - val_loss: 1.9662
Epoch 2/2
1500/1500 [==============================] - 266s 178ms/step - loss: 1.7893 - val_loss: 1.7473


In [36]:
#test_text = ['apparently they used too much synthetic flavors that it just burns your tongue also theres too much oil  almost made me chok']
#test_text = ['this stuff is awesome  for best flavor boil it in water drain the water add spice packet and then add hot water']


In [55]:
print(train['text_no_punctuation'][568442])
test_text = ['this product is great  gives you so much energy and tastes great  try this cafe latte and all the other flavors and you will not be disappointed']

In [37]:
#seq2seq_Model = tf.keras.models.load_model('seq2seq_subsample_1_epochs.h5')
#seq2seq_Model = tf.keras.models.load_model('seq2seq_full_data_3_epochs.h5')

In [38]:
#max_len_title = 30
# get the encoder's features for the decoder
tok1.fit_on_texts(test_text)

In [39]:
raw_tokenized = tok1.texts_to_sequences(test_text)
raw_tokenized = tf.keras.preprocessing.sequence.pad_sequences(raw_tokenized, maxlen=maxlen1)

In [40]:
body_encoding = encoder_model.predict(raw_tokenized) #predict the encoder state of the new sentence

In [41]:
latent_dim = seq2seq_Model.get_layer('Decoder-Word-Embedding').output_shape[-1]

In [42]:
#remember the get layer method for getting the embedding (word clusters)
decoder_inputs = seq2seq_Model.get_layer('Decoder-Input').input 
dec_emb = seq2seq_Model.get_layer('Decoder-Word-Embedding')(decoder_inputs)
dec_bn = seq2seq_Model.get_layer('Decoder-Batchnorm-1')(dec_emb)


In [43]:
gru_inference_state_input = tf.keras.Input(shape=(latent_dim,), name='hidden_state_input')



In [44]:
gru_out, gru_state_out = seq2seq_Model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input])

In [45]:
# Reconstruct dense layers
dec_bn2 = seq2seq_Model.get_layer('Decoder-Batchnorm-2')(gru_out)
dense_out = seq2seq_Model.get_layer('Final-Output-Dense')(dec_bn2)

In [46]:
decoder_model = tf.keras.Model([decoder_inputs, gru_inference_state_input],
                          [dense_out, gru_state_out])

In [47]:
# we want to save the encoder's embedding before its updated by decoder
#   because we can use that as an embedding for other tasks.
original_body_encoding = body_encoding

In [48]:
state_value = np.array(tok2.word_index['_start_']).reshape(1, 1)

In [49]:
state_value

array([[1]])

In [50]:
decoded_sentence = []
stop_condition = False

In [51]:
vocabulary_inv = dict((v, k) for k, v in tok2.word_index.items())
#vocabulary_inv[0] = "<PAD/>"
#vocabulary_inv[1] = "unknown"

In [52]:
vocabulary_inv

{1: '_start_',
 2: '_end_',
 3: 'great',
 4: 'good',
 5: 'the',
 6: 'coffee',
 7: 'best',
 8: 'love',
 9: 'tea',
 10: 'for',
 11: 'product',
 12: 'and',
 13: 'it',
 14: 'delicious',
 15: 'my',
 16: 'a',
 17: 'not',
 18: 'this',
 19: 'taste',
 20: 'yummy',
 21: 'excellent',
 22: 'very',
 23: 'dog',
 24: 'price',
 25: 'flavor',
 26: 'i',
 27: 'of',
 28: 'is',
 29: 'to',
 30: 'these',
 31: 'tasty',
 32: 'stuff',
 33: 'but',
 34: 'ever',
 35: 'favorite',
 36: 'yum',
 37: 'like',
 38: 'snack',
 39: 'food',
 40: 'awesome',
 41: 'loves',
 42: 'in',
 43: 'too',
 44: 'chocolate',
 45: 'wonderful',
 46: 'dogs',
 47: 'chips',
 48: 'as',
 49: 'so',
 50: 'them',
 51: 'treats',
 52: 'just',
 53: 'are',
 54: 'nice',
 55: 'hot',
 56: 'healthy',
 57: 'free',
 58: 'with',
 59: 'tasting',
 60: 'cookies',
 61: 'buy',
 62: 'treat',
 63: 'quality',
 64: 'tastes',
 65: 'perfect',
 66: 'better',
 67: 'you',
 68: 'on',
 69: 'sweet',
 70: 'value',
 71: 'at',
 72: 'green',
 73: 'really',
 74: 'what',
 75: 'no',


In [53]:
while not stop_condition:
    #print(1)
    preds, st = decoder_model.predict([state_value, body_encoding])

    pred_idx = np.argmax(preds[:, :, 2:]) + 2
    pred_word_str = vocabulary_inv[pred_idx]
    print(pred_word_str)
    if pred_word_str == '_end_' or len(decoded_sentence) >= maxlen2:
        stop_condition = True
        break
    decoded_sentence.append(pred_word_str)

    # update the decoder for the next word
    body_encoding = st
    state_value = np.array(pred_idx).reshape(1, 1)
    #print(state_value)

great
product
_end_


In [54]:
train.tail()

Summary  ...                                text_no_punctuation
568439                           a-ok  ...  we need this for a recipe my wife is intereste...
568442               Great Cafe Latte  ...  this product is great  gives you so much energ...
568448  Very large ground spice jars.  ...  my only complaint is that theres so much of it...
568449            Will not do without  ...  great for sesame chickenthis is a good if not ...
568453                    Great Honey  ...  i am very satisfied product is as advertised i...

[5 rows x 8 columns]